# Computer Vision Manipulations

In [2]:
import cv2
import numpy as np
import itertools as it
import os
import sys
import multiprocessing as mp
import pandas as pd

## Functions

In [3]:
def createHash(img, bot, top):
    hashmap = {}
    for i,j in it.product(range(img.shape[0]), range(img.shape[1])):
        if img[i,j] not in hashmap:
            if img[i,j] <= bot:
                hashmap[img[i,j]] = 0
            elif img[i,j] > top:
                hashmap[img[i,j]] = 255
            else:
                hashmap[img[i,j]] = 255 * ((img[i,j] - bot) / (top - bot))
                
    return hashmap

def grayscale(img):
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [4]:
def linearStretch(img):
    bot = min(img.ravel())
    top = max(img.ravel())
        
    hashmap = createHash(img, bot, top)
    img2 = img
    for i, j in it.product(range(img2.shape[0]), range(img2.shape[1])):
        img2[i,j] = hashmap[img2[i,j]]
        
    return img2

def createROI(img, h1, h2, w1, w2):
    
    return img[h1:h2, w1:w2]

def powerStretch(img):
    img2 = img
    #img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)  
    img2 = img2/255.0      
    im_power_law_transformation = cv2.pow(img2,2.5)
    im_power_law_transformation *= 255 
    im_power_law_transformation = im_power_law_transformation.astype('uint8')
    #im_power_law_transformation = cv2.cvtColor(im_power_law_transformation,cv2.COLOR_BGR2RGB)
        
    return im_power_law_transformation

def integratedColorModel(img):
    hsvimg = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)    
    hue = hsvimg[:,:,0]
    saturation = hsvimg[:,:,1]
    value = hsvimg[:,:,2]
    valuecs = linearStretch(value)
    saturationcs = linearStretch(saturation)
    hlscs = cv2.merge((hue,saturationcs,valuecs))
    imc = cv2.cvtColor(hlscs, cv2.COLOR_HSV2RGB)
   
    return imc

In [5]:
def getImagesPath(parent_path, child_path, filenames):
    paths = []
    for f in filenames:
        folder = f[:f.find('frame')-1]
        framename = 'wb_' + f[f.find('frame'):]
        if os.path.exists(parent_path + folder + child_path + framename):
            paths.append((folder,framename))
        
    return paths       

In [6]:
parent_path = '/home/diegues/2018-05-01-OMARE/logs/lauv-noptilus-3/20180501/'
child_path = '/mra/FilteredPhotos/WhiteBalance/'

df_path = "/home/diegues/Desktop/ProcessedImages/total_data.csv"
data = pd.read_csv(df_path)

filenames = data['filename']
image_paths = getImagesPath(parent_path, child_path, filenames)
image_paths[:5]

[('104728_cam-np3', 'wb_frame5.jpg'),
 ('104728_cam-np3', 'wb_frame9.jpg'),
 ('104728_cam-np3', 'wb_frame50.jpg'),
 ('104728_cam-np3', 'wb_frame51.jpg'),
 ('104728_cam-np3', 'wb_frame52.jpg')]

In [7]:
cv_path = '/'.join(df_path.split('/')[:-1]) + '/CV/'
icm_path = '/icm/'
lcs_path = '/lcs/'
pcs_path = '/pcs/'
if not os.path.exists(cv_path):
    os.mkdir(cv_path)
for folder in [f[0] for f in image_paths]:
    if not os.path.exists(cv_path + folder):
        os.mkdir(cv_path + folder)
        os.mkdir(cv_path + folder + icm_path)
        #os.mkdir(cv_path + folder + lcs_path)
        #os.mkdir(cv_path + folder + pcs_path)


In [13]:
def createImages(t):
    img = cv2.imread(parent_path + t[0] + child_path + t[1])
    print('Processing', t)
    icm = integratedColorModel(img)
    #lcs = linearStretch(grayscale(img))
    #pcs = powerStretch(grayscale(img))
    cv2.imwrite(cv_path + t[0] + icm_path + t[0] + t[1][t[1].find('_'):], icm)
    #cv2.imwrite(cv_path + t[0] + lcs_path + t[0] + t[1][t[1].find('_'):], lcs)
    #cv2.imwrite(cv_path + t[0] + pcs_path + t[0] + t[1][t[1].find('_'):], pcs)

In [14]:
pool = mp.Pool(processes=mp.cpu_count())
pool.map(createImages, image_paths)

Processing ('132143_forcadinho-np3', 'wb_frame2002.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1336.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame615.jpg')
Processing ('104728_cam-np3', 'wb_frame5.jpg')
Processing ('104728_cam-np3', 'wb_frame9.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1337.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2003.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame616.jpg')
Processing ('104728_cam-np3', 'wb_frame50.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1338.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2004.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame617.jpg')
Processing ('104728_cam-np3', 'wb_frame51.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame618.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2005.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1339.jpg')
Processing ('104728_cam-np3', 'wb_frame52.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2006.jpg')
Processing ('132143_forca

Processing ('132143_forcadinho-np3', 'wb_frame732.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1373.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame11.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2040.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame733.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1374.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame12.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2041.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame734.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1375.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame13.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2042.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame735.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1376.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame14.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame736.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2043.jpg')
Processing ('132143_forcadinho-np3', 'wb_fra

Processing ('132143_forcadinho-np3', 'wb_frame65.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2114.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1412.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame803.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame66.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2115.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1413.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame804.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame67.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2116.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1414.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame805.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame68.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2117.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1415.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame806.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame69.jpg')
Processing ('132143_forcadinho-np3', 'wb_fram

Processing ('132143_forcadinho-np3', 'wb_frame2254.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame101.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1449.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame861.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2265.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame102.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1450.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame862.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2325.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame103.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1451.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame863.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2344.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame104.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1452.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame864.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2345.jpg')
Processing ('132143_forcadinho-np3', 'w

Processing ('132143_forcadinho-np3', 'wb_frame143.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame904.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1484.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2489.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame144.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame905.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1485.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2490.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame145.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame906.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1486.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2491.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame146.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame907.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame1487.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame2493.jpg')
Processing ('132143_forcadinho-np3', 'wb_frame147.jpg')
Processing ('132143_forcadinho-np3', 'wb

...

Processing ('113610_cam_survey_1', 'wb_frame899.jpg')
Processing ('113610_cam_survey_1', 'wb_frame340.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1364.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2712.jpg')
Processing ('113610_cam_survey_1', 'wb_frame900.jpg')
Processing ('113610_cam_survey_1', 'wb_frame341.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1365.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2713.jpg')
Processing ('113610_cam_survey_1', 'wb_frame901.jpg')
Processing ('113610_cam_survey_1', 'wb_frame342.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1366.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2817.jpg')
Processing ('113610_cam_survey_1', 'wb_frame902.jpg')
Processing ('113610_cam_survey_1', 'wb_frame343.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1367.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2818.jpg')
Processing ('113610_cam_survey_1', 'wb_frame903.jpg')
Processing ('113610_cam_survey_1', 'wb_frame344.jpg')
Processing ('113610_

Processing ('113610_cam_survey_1', 'wb_frame1410.jpg')
Processing ('113610_cam_survey_1', 'wb_frame937.jpg')
Processing ('113610_cam_survey_1', 'wb_frame378.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2860.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1411.jpg')
Processing ('113610_cam_survey_1', 'wb_frame938.jpg')
Processing ('113610_cam_survey_1', 'wb_frame379.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2861.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1412.jpg')
Processing ('113610_cam_survey_1', 'wb_frame939.jpg')
Processing ('113610_cam_survey_1', 'wb_frame380.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2862.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1413.jpg')
Processing ('113610_cam_survey_1', 'wb_frame940.jpg')
Processing ('113610_cam_survey_1', 'wb_frame381.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1414.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2863.jpg')
Processing ('113610_cam_survey_1', 'wb_frame941.jpg')
Processing ('113610

Processing ('113610_cam_survey_1', 'wb_frame1448.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2897.jpg')
Processing ('113610_cam_survey_1', 'wb_frame974.jpg')
Processing ('113610_cam_survey_1', 'wb_frame416.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1449.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2898.jpg')
Processing ('113610_cam_survey_1', 'wb_frame975.jpg')
Processing ('113610_cam_survey_1', 'wb_frame417.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1450.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2899.jpg')
Processing ('113610_cam_survey_1', 'wb_frame976.jpg')
Processing ('113610_cam_survey_1', 'wb_frame418.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1451.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2900.jpg')
Processing ('113610_cam_survey_1', 'wb_frame977.jpg')
Processing ('113610_cam_survey_1', 'wb_frame419.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1458.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2901.jpg')
Processing ('11361

Processing ('113610_cam_survey_1', 'wb_frame1507.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2953.jpg')
Processing ('113610_cam_survey_1', 'wb_frame453.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1012.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1508.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2954.jpg')
Processing ('113610_cam_survey_1', 'wb_frame454.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1013.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1509.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2955.jpg')
Processing ('113610_cam_survey_1', 'wb_frame455.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1014.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1510.jpg')
Processing ('113610_cam_survey_1', 'wb_frame456.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2956.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1015.jpg')
Processing ('113610_cam_survey_1', 'wb_frame1511.jpg')
Processing ('113610_cam_survey_1', 'wb_frame2957.jpg')
Processing ('1

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [60]:
t = image_paths[0]
cv_path + t[0] + icm_path + t[0] + t[1][t[1].find('_'):]

'/home/diegues/Desktop/ProcessedImages/CV/113610_cam_survey_1/icm/113610_cam_survey_1_frame2013.jpg'